In [1]:
import cobra
import pandas as pd
import json
from copy import deepcopy
from custom_functions_scRBA import *

In [2]:
model = cobra.io.load_json_model('./models/cobraModel_constantStoichOnly.json')

with open('./models/symbolicStoich_lower_0_211.json') as f:
    coeffs_low = json.load(f)
with open('./models/symbolicStoich_higher_0_211.json') as f:
    coeffs_high = json.load(f)
    
coeffs_low = {tuple(k.split(' | ')):v for k,v in coeffs_low.items()}
coeffs_high = {tuple(k.split(' | ')):v for k,v in coeffs_high.items()}

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-26


In [3]:
model.solver = 'cplex'
model.solver.configuration.tolerances.feasibility = 1e-9
model.solver.configuration.tolerances.optimality = 1e-7

In [4]:
model.reactions.get_by_id('RXN-BTDDH_c_FWD-YAL060W').reaction

'MET-btd_c + MET-nad_c --> MET-actn__R_c + MET-h_c + MET-nadh_c'

In [5]:
mu = 0.1

if mu < 0.211:
    coeffs = deepcopy(coeffs_low)
else:
    coeffs = deepcopy(coeffs_high)
coeffs = {k:eval(str(v)) for k,v in coeffs.items()}

for k,v in coeffs.items():
    r,m = k
    rxn = model.reactions.get_by_id(r)
    met = model.metabolites.get_by_id(m)
    rxn.add_metabolites({met:v})

In [6]:
model.reactions.get_by_id('RXN-BTDDH_c_FWD-YAL060W').reaction

'4.213459e-07 ENZ-YAL060W + MET-btd_c + MET-nad_c --> MET-actn__R_c + MET-h_c + MET-nadh_c'

In [8]:
df_rxns = pd.read_excel('../scRBA_new/scRAM_network_frompy.xlsx')

In [9]:
df_rxns[df_rxns.id.str.contains('HEX1_c')]

,id,type,symbolic_coeffs,reaction
539,RXN-HEX1_c_FWD-YGL253W,metabolic,mu/keff,MET-atp_c + MET-glc__D_c + mu/keff ENZ-YGL253W...
540,RXN-HEX1_c_FWD-YLR446W,metabolic,mu/keff,MET-atp_c + MET-glc__D_c + mu/keff ENZ-YLR446W...
541,RXN-HEX1_c_FWD-YFR053C,metabolic,mu/keff,MET-atp_c + MET-glc__D_c + mu/keff ENZ-YFR053C...
542,RXN-HEX1_c_FWD-YCL040W,metabolic,mu/keff,MET-atp_c + MET-glc__D_c + mu/keff ENZ-YCL040W...


In [10]:
model.reactions.get_by_id('RXN-HEX1_c_FWD-YGL253W')

Reaction identifier,RXN-HEX1_c_FWD-YGL253W
Name,
Memory address,0x07f7aa67679d0
Stoichiometry,3.634418e-07 ENZ-YGL253W + MET-atp_c + MET-glc__D_c --> MET-adp_c + MET-g6p_c + MET-h_c 3.634418e-07 + + --> + +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
model.reactions.get_by_id('ENZSYN-YGL253W')

Reaction identifier,ENZSYN-YGL253W
Name,
Memory address,0x07f7aa6a4d190
Stoichiometry,2.0 PRO-YGL253W --> ENZ-YGL253W 2.0 -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
df_rxns[df_rxns.id.str.contains('ENZSYN-YGL253W')]

,id,type,symbolic_coeffs,reaction
3577,ENZSYN-YGL253W,enzyme,NaN,2 PRO-YGL253W --> ENZ-YGL253W


In [10]:
for k,v in coeffs.items():
    r,m = k
    rxn = model.reactions.get_by_id(r)
    met = model.metabolites.get_by_id(m)
    rxn.subtract_metabolites({met:v})

In [11]:
model.reactions.get_by_id('RXN-BTDDH_c_FWD-YAL060W').reaction

'MET-btd_c + MET-nad_c --> MET-actn__R_c + MET-h_c + MET-nadh_c'

#### Test build time

In [22]:
import time

Initial load

In [24]:
start_time = time.time()
model = cobra.io.load_json_model('./models/cobraModel_constantStoichOnly.json')

with open('./models/symbolicStoich_lower_0_211.json') as f:
    coeffs_low = json.load(f)
with open('./models/symbolicStoich_higher_0_211.json') as f:
    coeffs_high = json.load(f)
    
coeffs_low = {tuple(k.split(' | ')):v for k,v in coeffs_low.items()}
coeffs_high = {tuple(k.split(' | ')):v for k,v in coeffs_high.items()}

print('Run time:', (time.time() - start_time), 'sec')

Run time: 5.349748373031616 sec


Evaluate symbolic coefficients, add to model, then remove it to return to initial state

In [25]:
start_time = time.time()

mu = 0.1

if mu < 0.211:
    coeffs = deepcopy(coeffs_low)
else:
    coeffs = deepcopy(coeffs_high)
coeffs = {k:eval(str(v)) for k,v in coeffs.items()}

for k,v in coeffs.items():
    r,m = k
    rxn = model.reactions.get_by_id(r)
    met = model.metabolites.get_by_id(m)
    rxn.add_metabolites({met:v})
    
for k,v in coeffs.items():
    r,m = k
    rxn = model.reactions.get_by_id(r)
    met = model.metabolites.get_by_id(m)
    rxn.subtract_metabolites({met:v})
    
print('Run time:', (time.time() - start_time), 'sec')

Run time: 8.139089107513428 sec
